<a href="https://colab.research.google.com/github/Arnxmn/airlines_use_case/blob/main/airlines_claims_fake_data_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import random
from datetime import datetime, timedelta

# Define the Claim Types and Operation Segments
claim_types = {
    "Baggage-Related Issues": [
        "Lost or Delayed Baggage",
        "Damaged Baggage",
        "Overweight or Oversize Baggage Fees"
    ],
    "Flight Delays and Cancellations": [
        "Compensation for Delayed or Canceled Flights",
        "Reimbursement for Missed Connections",
        "Hotel Accommodations and Meal Vouchers"
    ],
    "Overbooking and Denied Boarding": [
        "Compensation for Involuntary Denied Boarding"
    ],
    "In-Flight Issues": [
        "Medical Emergencies",
        "Uncomfortable Seating",
        "Poor In-Flight Service"
    ],
    "Customer Service Issues": [
        "Rude or Unhelpful Staff",
        "Difficulty Resolving Complaints"
    ],
    "Other Issues": [
        "Mileage Disputes",
        "Upgrade Issues",
        "Overcharging",
        "Hidden Fees"
    ]
}

# Customer Segments
customer_segments = ["Basic", "Gold", "Platinum", "VIP"]
customer_segment_distribution = [0.57, 0.33, 0.08, 0.02]

# National vs International flight
flight_segments = ["National", "International"]
flight_segment_distribution = [0.7, 0.3]

# Claim_Type distribution based on Customer Segment
segment_claim_distribution = {
    "Basic": {
        "Flight Delays and Cancellations": 0.4,
        "Customer Service Issues": 0.2,
        "Baggage-Related Issues": 0.2,
        "In-Flight Issues": 0.1,
        "Overbooking and Denied Boarding": 0.05,
        "Other Issues": 0.05
    },
    "Gold": {
        "Flight Delays and Cancellations": 0.45,
        "Customer Service Issues": 0.2,
        "Baggage-Related Issues": 0.15,
        "In-Flight Issues": 0.1,
        "Overbooking and Denied Boarding": 0.05,
        "Other Issues": 0.05
    },
    "Platinum": {
        "Flight Delays and Cancellations": 0.5,
        "Customer Service Issues": 0.15,
        "Baggage-Related Issues": 0.1,
        "In-Flight Issues": 0.1,
        "Overbooking and Denied Boarding": 0.05,
        "Other Issues": 0.1
    },
    "VIP": {
        "Flight Delays and Cancellations": 0.5,
        "Customer Service Issues": 0.1,
        "Baggage-Related Issues": 0.1,
        "In-Flight Issues": 0.1,
        "Overbooking and Denied Boarding": 0.1,
        "Other Issues": 0.1
    }
}

# Generate a date within the 3-month timeframe
def generate_date(start_date, end_date):
    delta = end_date - start_date
    random_days = random.randint(0, delta.days)
    return start_date + timedelta(days=random_days)

In [ ]:
# Generate varied descriptions based on claim type and operation segment
def generate_description(claim_type, operation_segment):
    base_descriptions = {
        "Lost or Delayed Baggage": [
            "Customer's baggage did not arrive with the flight and was delivered to their home two days later. They are seeking compensation for the delay.",
            "The baggage of the customer was delayed by over 24 hours, causing inconvenience and stress during their vacation. Compensation is requested.",
            "Customer reported their baggage was lost upon arrival and was found three days later. They are requesting a reimbursement for interim expenses."
        ],
        "Damaged Baggage": [
            "The suitcase arrived with a broken wheel and a torn handle. The customer is requesting compensation for the damages.",
            "Customer found that their luggage was significantly scratched and dented upon arrival, affecting its usability.",
            "A large crack was found on the customer’s luggage upon retrieval. They are seeking reimbursement for repairs."
        ],
        "Overweight or Oversize Baggage Fees": [
            "Customer was charged an overweight baggage fee despite claiming the bag was under the limit. They dispute the accuracy of the scale used.",
            "The customer was charged an additional fee for an oversized bag that they believe was within the allowed dimensions.",
            "Dispute over baggage weight led to an unexpected fee, which the customer is seeking to have refunded."
        ],
        "Compensation for Delayed or Canceled Flights": [
            "Flight was delayed for 8 hours due to technical issues, causing the customer to miss an important event. They are requesting compensation.",
            "Due to a cancellation, the customer had to reschedule their entire trip, leading to extra costs that they are now claiming.",
            "The flight was delayed by several hours, leading to a missed connection. The customer is requesting compensation for the inconvenience."
        ],
        "Reimbursement for Missed Connections": [
            "Customer missed their connecting flight due to an initial delay and had to stay overnight at their own expense. They are seeking reimbursement.",
            "The delayed flight caused the customer to miss their connection, leading to the purchase of a new ticket. They are requesting compensation.",
            "Customer missed their connection due to a flight delay and incurred additional travel costs, for which they are seeking reimbursement."
        ],
        "Hotel Accommodations and Meal Vouchers": [
            "Customer's flight was canceled, but no accommodation or meal vouchers were provided. They are seeking compensation for the expenses incurred.",
            "Due to a long delay, the customer had to stay overnight but was not provided with hotel accommodations. They are requesting compensation.",
            "The flight cancellation forced the customer to book a hotel at their own expense, as no vouchers were offered. Compensation is requested."
        ],
        "Compensation for Involuntary Denied Boarding": [
            "Customer was involuntarily bumped from a flight due to overbooking and is seeking compensation for the inconvenience.",
            "The airline overbooked the flight, resulting in the customer being denied boarding. They are requesting compensation as per regulations.",
            "Customer was denied boarding due to overbooking, causing them to miss an important event. They are claiming compensation."
        ],
        "Medical Emergencies": [
            "Customer experienced a medical emergency during the flight, but reported inadequate response from the flight crew.",
            "A medical incident occurred mid-flight, but the customer felt the crew was not properly trained to handle it. They are filing a complaint.",
            "During the flight, the customer had a medical issue, and they are dissatisfied with the assistance provided by the crew."
        ],
        "Uncomfortable Seating": [
            "Customer reported that their seat was broken and could not recline, making the long flight extremely uncomfortable.",
            "The seat was very uncomfortable due to malfunctioning padding and limited legroom, causing significant discomfort during the flight.",
            "Customer's seat was stuck in an upright position for the entire flight, leading to a very unpleasant experience."
        ],
        "Poor In-Flight Service": [
            "Customer felt the in-flight service was poor, with inattentive staff and substandard meals.",
            "The in-flight attendants were unresponsive to requests, leading to a frustrating experience for the customer.",
            "Customer reported that the meal served was of very low quality, and the staff was not accommodating when they raised the issue."
        ],
        "Rude or Unhelpful Staff": [
            "Customer encountered rude behavior from the check-in staff, who were dismissive of their concerns.",
            "The gate agent was unhelpful and dismissive when the customer asked about their delayed flight.",
            "Customer reported that the airline's customer service representative was rude and uncooperative when addressing their issue."
        ],
        "Difficulty Resolving Complaints": [
            "Customer faced significant delays and frustration in resolving their complaint through customer service channels.",
            "Despite multiple attempts, the customer has not been able to resolve their complaint and is dissatisfied with the support received.",
            "The process to resolve a previous issue was complicated and ineffective, leading to further dissatisfaction."
        ],
        "Mileage Disputes": [
            "Customer noticed that a recent flight did not credit their mileage account correctly and is disputing the missing miles.",
            "Discrepancies in the customer’s mileage balance have led them to file a complaint requesting a review.",
            "The customer’s frequent flyer miles were not properly credited after their last flight, and they are requesting correction."
        ],
        "Upgrade Issues": [
            "Customer's upgrade request was mishandled, resulting in them being seated in a lower class than they paid for.",
            "The airline failed to process the customer's upgrade correctly, and they were not seated in the expected class.",
            "Customer paid for an upgrade but did not receive it upon boarding. They are seeking a refund for the upgrade fee."
        ],
        "Overcharging": [
            "Customer was overcharged for their flight ticket and is requesting a refund for the extra amount billed.",
            "The final price charged was higher than the amount shown during booking, leading the customer to request a refund.",
            "Customer discovered an unexpected charge on their ticket and is disputing the overcharge."
        ],
        "Hidden Fees": [
            "Customer was surprised by hidden fees in their ticket price and is requesting a refund for the undisclosed charges.",
            "Additional fees were added to the customer’s booking without clear explanation, prompting a refund request.",
            "Customer found hidden fees in their final bill, which were not explained at the time of booking."
        ]
    }

    return random.choice(base_descriptions[claim_type])


In [ ]:
# Function to generate claims data with specified trends
def generate_claims_data(num_records, start_date=datetime(2024, 6, 1), end_date=datetime(2024, 8, 31)):
    claims_data = []
    customer_id_list = list(range(10000, 10000 + num_records))
    random.shuffle(customer_id_list)

    for i in range(num_records):
        claim_id = i + 1
        customer_id = customer_id_list[i]
        flight_number = f"LA{random.randint(100, 999)}"
        incident_date = generate_date(start_date, end_date)
        claim_date = incident_date + timedelta(days=random.randint(0, 7))

        customer_segment = random.choices(customer_segments, customer_segment_distribution)[0]
        flight_segment = random.choices(flight_segments, flight_segment_distribution)[0]

        # Determine Claim Type based on Customer Segment distribution
        claim_type_category = random.choices(list(segment_claim_distribution[customer_segment].keys()), list(segment_claim_distribution[customer_segment].values()))[0]
        operation_segment = random.choice(claim_types[claim_type_category])
        #claim_type = f"{claim_type_category}: {operation_segment}"
        claim_type = claim_type_category

        description_of_incident = generate_description(operation_segment, claim_type_category)

        compensation_requested = round(random.uniform(50000, 500000), 2) if random.random() > 0.005 else None
        compensation_offered = compensation_requested * random.uniform(0.5, 1) if compensation_requested else None

        status = random.choice(["Open", "Closed"])

        # Resolution time based on exponential distribution
        resolution_time_distribution = {
            "Baggage-Related Issues": 10,
            "Flight Delays and Cancellations": 12,
            "Overbooking and Denied Boarding": 8,
            "In-Flight Issues": 14,
            "Customer Service Issues": 10,
            "Other Issues": 5
        }
        mean_resolution_time = resolution_time_distribution[claim_type_category]
        resolution_time = min(int(np.random.exponential(scale=mean_resolution_time)), 30)
        resolution_date = claim_date + timedelta(days=resolution_time) if status == "Closed" else None

        # Customer contact details (masked for privacy)
        customer_contact_details = f"+56-9-{random.randint(10000000, 99999999)}"

        claim_record = {
            "Claim_ID": claim_id,
            "Customer_ID": customer_id,
            "Flight_Number": flight_number,
            "Incident_Date": incident_date.strftime("%Y-%m-%d"),
            "Claim_Date": claim_date.strftime("%Y-%m-%d"),
            "Claim_Type": claim_type,
            "Operation_Segment": operation_segment,
            "Description_of_Incident": description_of_incident,
            "Compensation_Requested": compensation_requested,
            "Compensation_Offered": compensation_offered,
            "Status": status,
            "Resolution_Date": resolution_date.strftime("%Y-%m-%d") if resolution_date else None,
            "Customer_Contact_Details": customer_contact_details,
            "Customer_Segment": customer_segment,
            "Flight_Segment": flight_segment
        }

        claims_data.append(claim_record)

    return pd.DataFrame(claims_data)

In [ ]:
# Generate 3000 rows of synthetic data
claims_df = generate_claims_data(3000)
claims_df.head()
# Save the DataFrame to a CSV file
#claims_df.to_csv("synthetic_claims_data_updated.csv", index=False)


,Claim_ID,Customer_ID,Flight_Number,Incident_Date,Claim_Date,Claim_Type,Operation_Segment,Description_of_Incident,Compensation_Requested,Compensation_Offered,Status,Resolution_Date,Customer_Contact_Details,Customer_Segment,Flight_Segment
0,1,10654,LA882,2024-08-10,2024-08-17,Customer Service Issues,Difficulty Resolving Complaints,Customer faced significant delays and frustrat...,484306.80,346221.217308,Open,None,+56-9-42269467,Gold,International
1,2,11319,LA539,2024-08-01,2024-08-04,Flight Delays and Cancellations,Hotel Accommodations and Meal Vouchers,"Customer's flight was canceled, but no accommo...",240525.46,121032.204934,Open,None,+56-9-16943997,Basic,National
2,3,11679,LA335,2024-08-19,2024-08-25,Baggage-Related Issues,Overweight or Oversize Baggage Fees,Customer was charged an overweight baggage fee...,108925.13,73566.779482,Closed,2024-09-23,+56-9-82888954,Basic,International
3,4,11944,LA678,2024-07-11,2024-07-18,Baggage-Related Issues,Lost or Delayed Baggage,The baggage of the customer was delayed by ove...,124528.68,83377.674872,Open,None,+56-9-99862087,Basic,International
4,5,10743,LA867,2024-08-08,2024-08-12,Flight Delays and Cancellations,Reimbursement for Missed Connections,The delayed flight caused the customer to miss...,269197.80,215136.243965,Closed,2024-08-17,+56-9-51984787,Basic,National


In [ ]:
# STart exploring using PLots
import plotly.graph_objects as go
import plotly.express as px

# Convert 'Incident_Date' and 'Resolution_Date' to datetime objects
claims_df['Incident_Date'] = pd.to_datetime(claims_df['Incident_Date'])
claims_df['Resolution_Date'] = pd.to_datetime(claims_df['Resolution_Date'])

# A. Check claims trends by type
# Group by claim date and count claims
daily_claims = claims_df.groupby(['Claim_Date','Claim_Type'])['Claim_ID'].count().reset_index()
daily_claims.rename(columns={'Claim_ID': 'Claim_Count'}, inplace=True)

# Create the line chart
fig_trend = px.bar(daily_claims, x='Claim_Date', y='Claim_Count', color='Claim_Type',facet_col="Claim_Type", facet_col_wrap=3, title='Claims Trends by Claim Type')
fig_trend.show()

In [ ]:
# B. For closed claims : Check distribution of claims by customer segment and days that toook to resolve the claim

# Calculate resolution days
claims_df['Resolution_Days'] = (claims_df['Resolution_Date'] - claims_df['Incident_Date']).dt.days

# Filter for closed claims
closed_claims = claims_df[claims_df['Status'] == 'Closed']

# Group by customer segment and resolution days
grouped_cls_claims = closed_claims.groupby(['Customer_Segment', 'Resolution_Days'])['Claim_ID'].count().reset_index()
grouped_cls_claims.rename(columns={'Claim_ID': 'Claim_Count'}, inplace=True)
grouped_cls_claims

,Customer_Segment,Resolution_Days,Claim_Count
0,Basic,0.0,8
1,Basic,1.0,19
2,Basic,2.0,26
3,Basic,3.0,39
4,Basic,4.0,41
...,...,...,...
119,VIP,20.0,1
120,VIP,23.0,1
121,VIP,34.0,1
122,VIP,36.0,1


In [ ]:
closed_claims.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1488 entries, 2 to 2999
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Claim_ID                  1488 non-null   int64         
 1   Customer_ID               1488 non-null   int64         
 2   Flight_Number             1488 non-null   object        
 3   Incident_Date             1488 non-null   datetime64[ns]
 4   Claim_Date                1488 non-null   object        
 5   Claim_Type                1488 non-null   object        
 6   Operation_Segment         1488 non-null   object        
 7   Description_of_Incident   1488 non-null   object        
 8   Compensation_Requested    1481 non-null   float64       
 9   Compensation_Offered      1481 non-null   float64       
 10  Status                    1488 non-null   object        
 11  Resolution_Date           1488 non-null   datetime64[ns]
 12  Customer_Contact_Details 

In [ ]:
# C. Create a cummulative trend to check how quick are the claims closed
# Plot
fig = go.Figure()

# Calculate and plot cumulative percentage
for segment in grouped_cls_claims['Customer_Segment'].unique():
  # Dataframe for each segment
  segment_data = grouped_cls_claims[grouped_cls_claims['Customer_Segment'] == segment]
  total_claims = segment_data['Claim_Count'].sum()
  segment_data['Cumulative_Percentage'] = (segment_data['Claim_Count'].cumsum() / total_claims) * 100

  fig.add_trace(go.Scatter(x=segment_data['Resolution_Days'], y=segment_data['Cumulative_Percentage'], mode='lines', name=segment))

# Update layout
fig.update_layout(
    title='Cumulative % of Closed Claims by Resolution Days',
    xaxis_title='Resolution Days',
    yaxis_title='% of Closed Claims',
    legend_title='Customer Segment',
    hovermode='x unified',
    width=900,
    height=600
)

# Add gridlines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

# Show the plot
fig.show()


In [ ]:
# Run describe() for metrics related to closed claims
closed_claims.groupby('Customer_Segment')[['Compensation_Requested', 'Compensation_Offered']].describe()

Compensation_Requested                                \
                                  count           mean            std   
Customer_Segment                                                        
Basic                             825.0  279386.192412  128224.887657   
Gold                              487.0  271770.089651  130221.764211   
Platinum                          135.0  278151.586889  133368.390438   
VIP                                34.0  285940.180588  121659.986583   

                                                                            \
                       min         25%         50%          75%        max   
Customer_Segment                                                             
Basic             50759.66  168365.890  276068.080  389353.6000  499535.96   
Gold              50342.34  169206.880  263199.540  383259.0650  499844.82   
Platinum          50456.66  175013.880  279962.610  390156.2050  496269.91   
VIP               51338.91  216119.815  320606.995  387122.0825  465727.02   

                 Compensation_Offered                                \
                                count           mean            std   
Customer_Segment                                                      
Basic                           825.0  206739.820346  103885.699732   
Gold                            487.0  202088.442420  106938.503220   
Platinum                        135.0  209664.686439  107644.095685   
VIP                              34.0  210196.463895   84741.850513   

                                                                             \
                           min            25%            50%            75%   
Customer_Segment                                                              
Basic             28902.441044  122717.316791  195380.952069  279732.527901   
Gold              30194.162396  117125.170535  187386.512740  277834.406617   
Platinum          28287.356356  132554.257885  198424.057542  278084.182634   
VIP               26912.524940  169468.829292  223106.148825  268411.171309   

                                 
                            max  
Customer_Segment                 
Basic             493802.356790  
Gold              482102.289227  
Platinum          464718.996067  
VIP               344620.501303

In [ ]:
# D. Create boxplot to check distribution of compensation offered
boxplt = px.box(closed_claims, x='Customer_Segment', y='Compensation_Offered', color='Claim_Type', title='Distribution of Compensation Offered by Customer Segment')
boxplt.show()

In [ ]:
# Create a boxplot to check distribution of compensation requested based on Operation Issue
boxplt2 = px.box(claims_df, x='Operation_Segment', y='Compensation_Requested', color='Flight_Segment', title='Distribution of Compensation requested by Operation segment')
boxplt2.show()